In [8]:
%matplotlib inline
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### What variables are important for teams to win?

In [9]:
team = pd.read_csv('data/baseball/team.csv')

In [10]:
team = team[team.year >= 1985]

In [11]:
team['w_pct'] = team.w / 162
team['avg'] = team.h / team.ab
team['2b_avg'] = team.double / team.ab
team['3b_avg'] = team.triple / team.ab
team['hr_avg'] = team.hr / team.ab
team['r_avg'] = team.r / team.ab
team['so_avg'] = team.so / (162*27)
team['era_adj'] = team.era / 27
team['e_avg'] = team.e / (162*27)
team['bb_avg'] = team.bb / team.ab

In [12]:
from sklearn.model_selection import train_test_split

ImportError: cannot import name 'comb'

In [13]:
clean = team[['w_pct','r_avg','avg','bb_avg','so_avg','era_adj','e_avg']]
clean.describe()

,w_pct,r_avg,avg,bb_avg,so_avg,era_adj,e_avg
count,888.000000,888.000000,888.000000,888.000000,888.000000,888.000000,888.000000
mean,0.493278,0.133540,0.261666,0.096405,0.236956,0.155800,0.025017
std,0.073345,0.014543,0.011896,0.012839,0.035185,0.020758,0.004576
min,0.265432,0.094146,0.226483,0.067148,0.129858,0.107778,0.012346
25%,0.442901,0.123301,0.253077,0.087541,0.211706,0.140370,0.021719
50%,0.493827,0.132741,0.261446,0.095332,0.236740,0.154444,0.024691
75%,0.549383,0.143036,0.269449,0.105379,0.259088,0.169630,0.028121
max,0.716049,0.179091,0.293993,0.140986,0.350937,0.236296,0.040924


In [14]:
y = clean.pop('w_pct')

In [15]:
X = clean

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

NameError: name 'train_test_split' is not defined

In [17]:
from sklearn.tree import DecisionTreeRegressor

ImportError: cannot import name 'logsumexp'

In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

ImportError: cannot import name 'comb'

In [ ]:
tree = DecisionTreeRegressor(max_depth=6, max_leaf_nodes=39, random_state=42)
tree.fit(X_train,y_train)

In [ ]:
for x in zip(X.columns,tree.feature_importances_):
    print(x)

In [ ]:
cross_val_score(tree, X_train, y_train, cv=5).mean()

In [ ]:
score = []
test_range = np.arange(38,45,1)
for nodes in test_range:
    tree = DecisionTreeRegressor(max_depth=6, max_leaf_nodes=nodes)
    tree.fit(X_train,y_train)
    score.append(cross_val_score(tree, X_train, y_train, cv=5).mean())
plt.plot(test_range,score)
plt.xlabel('depth')
plt.ylabel('r2_score')

#### Testing other models

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.svm import SVR

In [ ]:
lin = LinearRegression()
lin.fit(X_train,y_train)
results = lin.predict(X_test)
mean_squared_error(y_test, results)

In [ ]:
cross_val_score(lin, X_train, y_train, cv=5).mean()

In [ ]:
lin.coef_

In [ ]:
lscv = LassoCV(eps=0.004)
lscv.fit(X_train,y_train)
results = lscv.predict(X_test)
mean_squared_error(y_test, results)

In [ ]:
cross_val_score(lscv, X_train, y_train, cv=5).mean()

In [ ]:
lscv.coef_

In [ ]:
svr = SVR(C=1000,epsilon=0.039)
svr.fit(X_train,y_train)
results = svr.predict(X_test)
mean_squared_error(y_test, results)

In [ ]:
cross_val_score(svr, X_train, y_train, cv=5).mean()

In [ ]:
score = []
test_range = np.arange(0,0.05,0.001)
for e in test_range:
    svr = SVR(C=1000,epsilon=e)
    svr.fit(X_train,y_train)
    score.append(cross_val_score(svr, X_train, y_train, cv=5).mean())
plt.plot(test_range,score)
plt.xlabel('epsilon parameter')
plt.ylabel('mean_squared_error')

In [ ]:
score = []
test_range = np.arange(0.001,0.01,0.001)
for eps in test_range:
    lscv = LassoCV(eps=eps)
    lscv.fit(X_train,y_train)
    score.append(cross_val_score(lscv, X_train, y_train, cv=5).mean())
plt.plot(test_range,score)
plt.xlabel('n_alphas')
plt.ylabel('mean_squared_error')

### What is important for players?

In [ ]:
batting = pd.read_csv('data/baseball/batting.csv')
salary = pd.read_csv('data/baseball/salary.csv')

In [ ]:
batting = batting[(batting.year >= 1985) & (batting.ab > 100)]

In [ ]:
batting['avg'] = batting.h / batting.ab
batting['1b_avg'] = (batting.h - batting.double - batting.triple - batting.hr) / batting.ab
batting['2b_avg'] = batting.double / batting.ab
batting['3b_avg'] = batting.triple / batting.ab
batting['hr_avg'] = batting.hr / batting.ab
batting['r_avg'] = batting.r / batting.ab
batting['obp'] = ((batting.h + batting.bb + batting.hbp + batting.ibb) /
    (batting.ab + batting.bb + batting.hbp + batting.ibb))
batting['slug'] = (batting.h + batting.double + batting.triple * 2 + batting.hr * 3) / batting.ab
batting['bb_avg'] = (batting.bb + batting.ibb) / batting.ab
batting['hbp_avg'] = batting.hbp / batting.ab

In [ ]:
### OBP is the most important
clean_batting = batting[['r_avg','avg','obp','slug']].dropna()
clean_batting.describe()

In [ ]:
# What part of OBP is most important?
clean_batting = batting[['r_avg','1b_avg','2b_avg','3b_avg','hr_avg','bb_avg','hbp_avg']].dropna()
clean_batting.describe()

In [ ]:
y = clean_batting.pop('r_avg')

In [ ]:
X = clean_batting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
tree = DecisionTreeRegressor(max_depth=7, max_leaf_nodes=112, random_state=42)
tree.fit(X_train,y_train)

In [ ]:
for x in zip(X.columns,tree.feature_importances_):
    print(x)

In [ ]:
cross_val_score(tree, X_train, y_train, cv=5).mean()

In [ ]:
score = []
test_range = np.arange(103,114,1)
for nodes in test_range:
    tree = DecisionTreeRegressor(max_depth=7,max_leaf_nodes=nodes)
    tree.fit(X_train,y_train)
    score.append(cross_val_score(tree, X_train, y_train, cv=5).mean())
plt.plot(test_range,score)
plt.xlabel('max_leaf_nodes')
plt.ylabel('r2_score')